<a href="https://colab.research.google.com/github/chenboju/AI/blob/main/17_Bert%E4%BF%AE%E6%94%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 查看GPU

In [1]:
!nvidia-smi

Mon Apr 15 05:23:24 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### 安裝需要的BERT套件

In [2]:
!pip install transformers==4.29.0 -U
!pip install nlp -U
!pip install torch==2.0.0 -U
!pip install transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 56.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 65.8 MB/s eta 0:00:00
  

### 載入套件

In [3]:
from transformers import BertForSequenceClassification, BertTokenizerFast, Trainer, TrainingArguments
from nlp import load_dataset, Dataset
import nlp
import torch
import random
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

### 抓取訓練資料

In [4]:
!wget https://github.com/shhuangmust/AI/raw/112-1/train.csv

--2024-04-15 05:27:12--  https://github.com/shhuangmust/AI/raw/112-1/train.csv
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/shhuangmust/AI/112-1/train.csv [following]
--2024-04-15 05:27:12--  https://raw.githubusercontent.com/shhuangmust/AI/112-1/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 388895 (380K) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>] 379.78K  --.-KB/s    in 0.04s   

2024-04-15 05:27:13 (9.40 MB/s) - ‘train.csv’ saved [388895/388895]



### 讀取訓練資料集

In [5]:
df = pd.read_csv('train.csv')
dataset = Dataset.from_pandas(df)

In [6]:
# df.sample(10) #取10筆樣本
df.sample(20)

,ID,Name,Description,label
570,570,速養療、福寧補,「…記者陳O珍台北報導…癌症…化療副作用…罹癌後的種種病痛…提供癌症病患的營養補充，以作身體...,1
91,91,嫚妙纖_亮妍C_發泡錠、嫚妙_紅石榴_發泡錠_優惠組_亞瑪勁台灣旗艦館,「.7天有效排毒美顏...蚊子黑色素痘痘通通OUT!...7天有效淡斑亮顏...平衡酸性體質...,1
34,34,「Neocell二型膠原蛋白～關節保健～含玻尿酸、軟骨素120caps」、「美國代購～Ddr...,「健康結締組織、軟骨支持...2型膠原蛋白是軟骨組織中的關鍵結構蛋白，隨著年齡的增長而惡化....,1
830,830,康采膠原Q10隨身包avon,維持緊緻青春美麗膠原+Q10：每份添加2500mg小分子膠原蛋白及水溶性Q10，迅速吸收，陽...,0
905,905,https://tw.buy.yahoo.com/gdsale/GNC健安喜-孕養調理-葉酸...,為水溶性維他命B群中的成員之ㄧ。-每顆含有400微克的葉酸成分。-參與紅血球的形成，也參與核...,0
423,423,【FINEJAPAN】金亮Q10膠囊60粒、【FINEJAPAN】金亮綠咖啡孅盈錠150錠、...,「有效降低心血管疾病...促進免疫系統功能，改善過敏症狀…有效降低心血管，心臟病疾病…尋求預...,1
147,147,養生月子餐,「養生月子餐」「『五色入五臟』、『經絡可養生』…氣血耗損…肝、心、脾、肺、腎五臟經歷懷孕、生...,1
320,320,帝皇酵素,「帝皇酵素…產品成份：諾麗果、霍山石斛、田七、刺五加、靈芝、何首烏、肉蓯蓉、川貝母、天山雪蓮...,1
810,810,康采B群維他命-avon,B退疲累精神奕奕完整七種維生素B群：含有維生素B1、B2、B6、B12、菸鹼醯胺、葉酸、生物...,0
644,644,賀寶芙深海魚油膠囊,深海魚油膠囊含Omage-3脂肪酸及維生素E，提供重要的營養。,0


### 載入原始模型

In [7]:
model = BertForSequenceClassification.from_pretrained('hfl/chinese-bert-wwm-ext') #hugface 模型

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of the model checkpoint at hfl/chinese-bert-wwm-ext were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkp

###  載入分詞模型

In [8]:
#tokenizer = BertTokenizerFast.from_pretrained('hfl/chinese-bert-wwm-ext') #hugface 模型

tokenizer = BertTokenizerFast.from_pretrained('google-bert/bert-base-chinese')

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

### 設定執行參數

In [9]:
RANDOM_SEED = 5 #隨機5個
MAX_LEN = 512 #最大長度
EPOCHS=5  #5週期(5次)
BATCH_SIZE = 8 #每批次大小8

### 把資料集分成訓練、測試、驗證

In [11]:
# 使用隨機種子對資料集進行洗牌，以確保結果的可複現性
shuffled_ds = dataset.shuffle(RANDOM_SEED) #RANDOM_SEED=5

# 將洗牌後的資料集分割為訓練集和測試/驗證集
split_ds = shuffled_ds.train_test_split(test_size=0.2) #2成測試

# 從分割後的資料集中取出訓練集
train_dataset = split_ds['train'] #8成訓練
test_val_dataset = split_ds['test']

# 將測試/驗證集進一步分割為獨立的測試集和驗證集
split_tv = test_val_dataset.train_test_split(test_size=0.5) #5成測試

# 從分割後的資料集中取出測試集
test_dataset = split_tv['train'] #5成訓練

# 從分割後的資料集中取出驗證集
val_dataset = split_tv['test']


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

### 取一筆出來瞧瞧(已經打亂)

In [23]:
train_dataset[1]

{'label': tensor(1),
 'input_ids': tensor([  101,   519,   100,  7349,  7013,  7349,  4077,   100,   823,  3481,
          4495,  7015,  1399,  3563,  2130,  5401,  3289,  5221,  5401,  5270,
          7614,   924,  6349,  8125,  2207,  3229,  5544,  5505,   118, 10431,
           110,  8125,  2207,  3229,   807,  6342,   116,  9959,   110,  8108,
          1921,  7768,  7028,   118,   126,   100,  3952,  7431,  5544,  5505,
          7000,  1419,  7030,  3683,   671,  5663,   782,  4685,  3683,  3938,
          2208,  6818,  8230,   945,   100,  6868,  5445,  3122,  3080,   782,
          7768,  3918,  5993,  7432,   809,  4482,  6237,  4638,  3952,  7431,
          5544,  5505,   100,  2571,  4321,  3976,  4989,  1315,  1146,  6237,
          3938,  1397,   100,  6868,  5445,  6366,  6716,  7768,  2768,  4158,
          3211,  4607,  7768,  6549,   100,  8399,  8714,   118,  8264,  8714,
           100,  3867,  7370,  3952,  7431,  5544,  5505,   100,   520,   510,
           519,   

In [13]:
tokenizer.tokenize("體重跟體脂都有在持續緩慢的下降中")

['體',
 '重',
 '跟',
 '體',
 '脂',
 '都',
 '有',
 '在',
 '持',
 '續',
 '緩',
 '慢',
 '的',
 '下',
 '降',
 '中']

### 定義分詞器

In [14]:
# 定義一個函數 tokenize，該函數用於對資料進行分詞處理
def tokenize(batch):
    return tokenizer(batch['Description'], max_length=MAX_LEN, padding=True, truncation=True)

# 對訓練集進行分詞處理，使用 map 函數將 tokenize 函數應用到整個資料集上
# 設置 batched=True 表示對資料進行批處理，batch_size 設置為訓練集大小 len(train_dataset)
train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))

# 對測試集進行分詞處理，使用 map 函數將 tokenize 函數應用到整個資料集上
# 設置 batched=True 表示對資料進行批處理，batch_size 設置為測試集大小 len(test_dataset)
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))

# 對驗證集進行分詞處理，使用 map 函數將 tokenize 函數應用到整個資料集上
# 設置 batched=True 表示對資料進行批處理，batch_size 設置為驗證集大小 len(val_dataset)
val_dataset = val_dataset.map(tokenize, batched=True, batch_size=len(val_dataset))

# 將處理後的訓練集格式設置為 PyTorch 格式，指定要包含的資料列為 input_ids、attention_mask 和 label
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

# 將處理後的測試集格式設置為 PyTorch 格式，指定要包含的資料列為 input_ids、attention_mask 和 label
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

# 將處理後的驗證集格式設置為 PyTorch 格式，指定要包含的資料列為 input_ids、attention_mask 和 label
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

### 定義評測準則，訓練器

In [15]:
# 定義一個函數 compute_metrics 來計算模型評估指標
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# 定義訓練的相關參數
training_args = TrainingArguments(
    output_dir='./results',  # 設置訓練結果的輸出目錄
    num_train_epochs=EPOCHS,  # 設置訓練的 epoch 數量
    per_device_train_batch_size=BATCH_SIZE,  # 設置每個訓練裝置的批次大小
    per_device_eval_batch_size=BATCH_SIZE,  # 設置每個評估裝置的批次大小
    warmup_steps=50,  # 設置學習率的 warmup 步數
    weight_decay=0.01,  # 設置權重衰減的參數
    logging_dir='./logs',  # 設置日誌的輸出目錄
)

# 創建 Trainer 實例，用於訓練模型
trainer = Trainer(
    model=model,  # 指定要訓練的模型
    args=training_args,  # 指定訓練的參數
    compute_metrics=compute_metrics,  # 指定用於計算評估指標的函數
    train_dataset=train_dataset,  # 指定訓練資料集
    eval_dataset=test_dataset  # 指定評估資料集
)


### 開始訓練

In [16]:
trainer.train()
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


{'eval_loss': 0.0007229068432934582,
 'eval_accuracy': 1.0,
 'eval_f1': 1.0,
 'eval_precision': 1.0,
 'eval_recall': 1.0,
 'eval_runtime': 2.5142,
 'eval_samples_per_second': 37.388,
 'eval_steps_per_second': 4.773,
 'epoch': 5.0}

### 找一個範例來測試

In [17]:
print(df.iloc[220]['label'], df.iloc[220]['Description'])
print(df.iloc[578]['label'], df.iloc[578]['Description'])

1 「增強免疫力...護眼明目...抗氧化...保肝...皮膚：...保護皮膚，減緩皮屑問題，降低敏感...毛髮……促進毛髮亮麗...消化系統」「強化免疫系統，有助調整過敏體質...清除自由基延緩老化...調整腸胃道機能...調理皮膚問題...諾麗果含有許多強大的抗氧化劑」「阻止或減少病痛的發生，並迅速修補已受傷的細胞，將代謝殘渣或毒素排出體外」「它能維護身體細胞組織正常運作，阻止或減少病痛的發生，並迅速修補已受傷的細胞，將代謝殘渣或毒素排出體外，使身體恢復正常」
0 ·???????每份含有30mg輔Q10。輔Q10可參與維生素E再生，兩者具協同作用，可促進新陳代謝。保護循環順暢維持健康。暢銷日本的最佳美容聖品。


### 設定測試文字

In [41]:
test_text = [
    "???????每份含有30mg輔Q10。輔Q10可參與維生素E再生，兩者具協同作用，可促進新陳代謝。保護循環順暢維持健康。暢銷日本的最佳美容聖品",
    "想要無限的能量？每天服用我們的 '超能力夢幻丸'，每粒含有3000mg的 '從未見過的無敵元素'，保證不會讓您成為外星人！",
    "活力無限，來自每份1000mg維生素C！強效抗氧化，守護您的免疫系統！",
    "活力來源，蘊藏在每份500mg鈣質中！強化骨骼，健康成長！",
    "保護您的心臟，每天服用500mg歐米加3！維持心血管健康，享受活力生活！",
    "我們的產品，像在臭鼬的世界中狂奔一樣令人不舒服！",
    "我們的新產品，每口都帶來500mg的 '無法置信的無敵保健元素'，無論您相信與否，它都不會讓您永遠睡著！",
    "男人喝了硬邦邦，鐵棒一樣非常有自信…女人喝了水汪汪…容易高潮…壯陽品…對男性能提高性功能..."
]

# 將廣告語組合成一個字串
combined_text = ' '.join(test_text)

# 設置最大序列長度為 512
MAX_LEN = 512

# 編碼輸入文本
encoded_review = tokenizer.encode_plus(
    # 需要編碼的原始文本字符串
    combined_text,

    # 設置最大序列長度
    max_length=MAX_LEN,

    # 添加特殊標記,如 [CLS] 和 [SEP]
    add_special_tokens=True,

    # 不需要返回 token 類型 id
    return_token_type_ids=False,

    # 返回注意力掩碼張量
    return_attention_mask=True,

    # 將返回的張量轉換為 PyTorch 張量
    return_tensors='pt',

    # 如果序列長度超過 max_length,將進行截斷
    truncation=True
)


In [42]:
encoded_review

{'input_ids': tensor([[ 101,  136,  136,  136,  136,  136,  136,  136, 3680,  819, 1419, 3300,
         8114, 8882, 6737,  100,  511, 6737,  100, 1377, 1347, 5645, 5204, 4495,
         5162,  100, 1086, 4495, 8024, 1060, 5442, 1072, 1295, 1398,  868, 4500,
         8024, 1377,  914, 6868, 3173, 7376,  807, 6342,  511,  924, 6362, 2542,
         4472, 7518, 3268, 5204, 2898,  978, 2434,  511, 3268, 7077, 3189, 3315,
         4638, 3297,  881, 5401, 2159, 5469, 1501, 2682, 6206, 4192, 7361, 4638,
         5543, 7030, 8043, 3680, 1921, 3302, 4500, 2769,  947, 4638,  112, 6631,
         5543, 1213, 1918, 2404,  709,  112, 8024, 3680, 5108, 1419, 3300, 8283,
         8882, 4638,  112, 2537, 3313, 6210, 6882, 4638, 4192, 3147, 1039, 5162,
          112, 8024,  924, 6349,  679, 3298, 6366, 2644, 2768, 4158, 1912, 3215,
          782, 8013, 3833, 1213, 4192, 7361, 8024,  889, 5632, 3680,  819, 8212,
         8882, 5204, 4495, 5162,  100, 8013, 2485, 3126, 2834, 3709, 1265, 8024,
         2127,

### 進行測試

In [43]:
# 將預訓練模型移動到 GPU 上
test_model = model.to('cuda')

# 定義類別名稱
classnames = ['優秀', '不優秀']

# 遍歷測試文本列表
for text in test_text:
    # 對當前測試文本進行編碼
    encoded_review = tokenizer.encode_plus(
        text,
        max_length=MAX_LEN,
        add_special_tokens=True,
        return_token_type_ids=False,
        return_attention_mask=True,
        return_tensors='pt',
        truncation=True
    )

    # 從編碼後的數據中獲取 input_ids 和 attention_mask
    # 並將它們移動到 GPU 上
    input_ids = encoded_review['input_ids'].to('cuda')
    attention_mask = encoded_review['attention_mask'].to('cuda')

    # 將編碼後的輸入喂給模型,獲取模型輸出
    output = test_model(input_ids, attention_mask)

    # 從模型輸出中找到分數最高的類別索引
    result = torch.argmax(output[0][0])

    # 打印原始測試文本以及模型對該文本的分類結果
    print(f'廣告詞: {text}')
    print(f'檢測結果: {classnames[result]}')

廣告詞: ???????每份含有30mg輔Q10。輔Q10可參與維生素E再生，兩者具協同作用，可促進新陳代謝。保護循環順暢維持健康。暢銷日本的最佳美容聖品
檢測結果: 優秀
廣告詞: 想要無限的能量？每天服用我們的 '超能力夢幻丸'，每粒含有3000mg的 '從未見過的無敵元素'，保證不會讓您成為外星人！
檢測結果: 優秀
廣告詞: 活力無限，來自每份1000mg維生素C！強效抗氧化，守護您的免疫系統！
檢測結果: 優秀
廣告詞: 活力來源，蘊藏在每份500mg鈣質中！強化骨骼，健康成長！
檢測結果: 優秀
廣告詞: 保護您的心臟，每天服用500mg歐米加3！維持心血管健康，享受活力生活！
檢測結果: 優秀
廣告詞: 我們的產品，像在臭鼬的世界中狂奔一樣令人不舒服！
檢測結果: 優秀
廣告詞: 我們的新產品，每口都帶來500mg的 '無法置信的無敵保健元素'，無論您相信與否，它都不會讓您永遠睡著！
檢測結果: 優秀
廣告詞: 男人喝了硬邦邦，鐵棒一樣非常有自信…女人喝了水汪汪…容易高潮…壯陽品…對男性能提高性功能...
檢測結果: 不優秀
